In [1]:
import json
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
import pickle   
from lifelines import CoxPHFitter # install lifelines-0.30.0 if not already installed
from lifelines.utils import concordance_index


In [2]:
# load test features
test_radiomics = pd.read_csv('D:\\Shruti\\IS_project\\test_radiomics_lung_tumor.csv')

# retain patient_id separately; you need the surival data for these patients also (survival time and event columns)
pids = test_radiomics['patient_id'].values
# survival_time = test_radiomics['survival_time'].values
# event = test_radiomics['event'].values
test_radiomics = test_radiomics.drop(columns=['patient_id'])

# remove features that dont have 'original' in their name
# keep_cols = [col for col in test_radiomics.columns if ('diagnostics' not in col)] # texture features
keep_cols = [col for col in test_radiomics.columns if ('diagnostics' not in col) and ('shape' in col)] # shape features only
test_radiomics = test_radiomics[keep_cols]  

# add relevant suffix to each column name, acc. to the ROI (check results.json file used)
test_radiomics.columns = [col + '_NSCLC' for col in test_radiomics.columns]
test_radiomics.head()

,original_shape_Elongation_NSCLC,original_shape_Flatness_NSCLC,original_shape_LeastAxisLength_NSCLC,original_shape_MajorAxisLength_NSCLC,original_shape_Maximum2DDiameterColumn_NSCLC,original_shape_Maximum2DDiameterRow_NSCLC,original_shape_Maximum2DDiameterSlice_NSCLC,original_shape_Maximum3DDiameter_NSCLC,original_shape_MeshVolume_NSCLC,original_shape_MinorAxisLength_NSCLC,original_shape_Sphericity_NSCLC,original_shape_SurfaceArea_NSCLC,original_shape_SurfaceVolumeRatio_NSCLC,original_shape_VoxelVolume_NSCLC
0,0.597984,0.278704,60.529228,217.181354,265.176304,177.179287,213.379481,281.557322,33323.424471,129.870905,0.410690,12193.717390,0.365920,33465.257904
1,0.536308,0.179125,49.928049,278.732760,217.567958,258.324451,179.751324,292.885624,8901.971397,149.486739,0.445607,4661.466064,0.523644,8991.883301
2,0.183543,0.128461,40.594035,316.002895,23.697089,65.336257,223.570926,248.480687,8135.962996,58.000006,0.496903,3936.879071,0.483886,8221.542864
3,0.547651,0.347226,54.924795,158.181779,237.867159,219.905528,216.694081,239.180912,154360.623671,86.628407,0.405002,34359.815322,0.222594,154525.542343
4,0.351964,0.251468,20.596042,81.903353,242.375265,90.488367,172.598323,264.417673,11225.827258,28.827071,0.500522,4844.023580,0.431507,11267.455964


In [3]:
results_json = json.load(open('D:\\Shruti\\IS_project\\models\\tumor_volume\\results.json'))
selected_features = results_json['selected_features']

# make new dataframe X_test from test_radiomics and retain patient_id_NSCLC column
X_test = test_radiomics.copy()

# filter only texture features (remove shape features)
f_names = X_test.columns.tolist()
shape_features = [f for f in f_names if 'shape' in f]
X_test = X_test[shape_features]

# scale test data using the same scalar used for training
with open('D:\\Shruti\\IS_project\\models\\mediastinal_nodes_volume\\scalar.pkl', 'rb') as f:
    scaler = pickle.load(f)
X_test = pd.DataFrame(scaler.transform(X_test.values), columns=X_test.columns)  


# select only the selected features from test_radiomics
X_test_selected = X_test[selected_features]

# check if the number of selected features matches the number of columns in X_test_selected
print(len(selected_features), len(X_test_selected.columns))

12 12


c:\Users\P70073858\.conda\envs\shruti_env\lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.5.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
# load CoxPH model from pickle file


with open('D:\\Shruti\\IS_project\\models\\tumor_volume\\coxph_model.pkl', 'rb') as f:
    cph = pickle.load(f)

# predict risk scores for test set
risk_scores = -cph.predict_partial_hazard(X_test_selected)

# calculate C-index --> please ensure you pass the correct survival time and event data for the test set patients here
test_c_index = concordance_index(event_observed=survival_time_column,  # --> edit as needed
                                 predicted_scores=risk_scores,
                                 event_times=event_column) # --> edit as needed
print("C-index on test set: ", test_c_index)

NameError: name 'survival_time_column' is not defined